# Imports and infrastructure

In [ ]:
# !pip install --upgrade transformers==4.49.0
# !pip install optuna==2.10.0
# !pip install numpy==1.26.4 gensim==4.3.2
# !pip install scipy==1.12.0
# !pip install --upgrade pandas==2.2.2
# !pip install h3
# !pip install mlflow
# !pip install 'protobuf<4'
# !pip install selenium
# !pip install natasha
# !pip install pymystem3
# !pip install symspellpy
!rm -rf /content/Price-prediction-with-textual-data

In [ ]:
!git clone https://github.com/anna-k-00/Price-prediction-with-textual-data.git

Cloning into 'Price-prediction-with-textual-data'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 359 (delta 32), reused 6 (delta 6), pack-reused 303 (from 2)
Receiving objects: 100% (359/359), 2.09 MiB | 18.11 MiB/s, done.
Resolving deltas: 100% (180/180), done.


In [ ]:
# Шаг 1: Проверка и настройка окружения
import os
import sys
import importlib

# Шаг 2: Клонирование/обновление репозитория
repo_url = 'https://github.com/anna-k-00/Price-prediction-with-textual-data.git'
repo_dir = 'Price-prediction-with-textual-data'

if not os.path.exists(repo_dir):
    !git clone {repo_url}
else:
    !cd {repo_dir} && git pull

# Шаг 3: Добавляем все нужные пути в sys.path
paths_to_add = [
    f'/content/{repo_dir}',                     # Для файлов в корне (parser_avito.py)
    f'/content/{repo_dir}/main_methods',        # Основные модули
    f'/content/{repo_dir}/embeddings_generation', # Генерация эмбеддингов
    f'/content/{repo_dir}/preprocessors'        # Препроцессоры
]

for path in paths_to_add:
    if os.path.exists(path) and path not in sys.path:
        sys.path.insert(0, path)
        print(f'Добавлен путь: {path}')

# Шаг 4: Собираем список всех модулей для импорта
all_modules = [
    # Основные модули
    'resource_monitor', 'ann', 'predict', 'test_pipeline',

    # Модули из embeddings_generation
    'embeddings_generation.rubert_transformer',
    'embeddings_generation.tfidf_transformer',
    'embeddings_generation.w2v_transformer',
    'embeddings_generation.gate',

    # Модули из preprocessors
    'preprocessors.preprocessor_params_hex',
    'preprocessors.preprocessor_text',

    # Отдельные файлы в корне
    'parser_avito'
]

# Шаг 5: Импортируем все модули
imported_modules = {}
failed_modules = {}

for module_name in all_modules:
    try:
        module = importlib.import_module(module_name)
        imported_modules[module_name] = module
        print(f'✅ {module_name} успешно импортирован')
    except Exception as e:
        failed_modules[module_name] = str(e)
        print(f'❌ Ошибка импорта {module_name}: {str(e)[:200]}')  # Обрезаем длинные сообщения

Already up to date.
Добавлен путь: /content/Price-prediction-with-textual-data
Добавлен путь: /content/Price-prediction-with-textual-data/main_methods
Добавлен путь: /content/Price-prediction-with-textual-data/embeddings_generation
Добавлен путь: /content/Price-prediction-with-textual-data/preprocessors
✅ resource_monitor успешно импортирован
✅ ANN успешно импортирован
✅ predict успешно импортирован


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✅ test_pipeline успешно импортирован
✅ embeddings_generation.rubert_fine_tuning успешно импортирован
✅ embeddings_generation.tfidf_generator успешно импортирован
✅ embeddings_generation.w2v_generator успешно импортирован
✅ embeddings_generation.gate успешно импортирован
✅ preprocessors.preprocessor_params_hex успешно импортирован
✅ preprocessors.preprocessor_text успешно импортирован
✅ parser_avito успешно импортирован


In [ ]:
import logging
import os
import time
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
import mlflow
import mlflow.sklearn
import warnings
from transformers import AutoModel, AutoTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.svm import LinearSVR
from sklearn.base import BaseEstimator, TransformerMixin
import random
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import torch.nn as nn
import os
import joblib
import transformers
import json
import joblib
from datetime import datetime
from sklearn.decomposition import PCA
import time
import sys
import platform
import psutil
import threading

try:
    import pynvml
    pynvml.nvmlInit()
    HAS_NVML = True
except Exception:
    HAS_NVML = False

from ANN import ANNRegressor
from resource_monitor import ResourceMonitor
from test_pipeline import PricePredictionExperiment
from preprocessor_params_hex import DataProcessingPipeline

from tfidf_transformer import TfidfTransformer
from w2v_transformer import Word2VecTransformer
from rubert_transformer import RuBertTiny2Embedder

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# mkdir -p "/content/drive/My Drive/mlflow_data_DROP"

In [ ]:
# set your ml flow directory

mlflow.set_tracking_uri('file:///content/drive/My Drive/mlflow_data_DROP')

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

# Initiation for fast checking

### XGBR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    xgbr_params = {
        'model__n_estimators': [500, 1000],
        'model__max_depth': [4, 6],
        'model__learning_rate': [0.005, 0.01, 0.02, 0.05],
        'model__subsample': [0.8, 1.0]
    }

    combinations = [
        ('categorical-only', None, False, False, True, True),
        ('categorical-only', None, False, False, False, True),
        ('mixed', 'rubert', True, False, False, True),
        ('mixed', 'w2v', True, False, False, True),
        ('mixed', 'tfidf', True, False, False, True),
        ('text-only', 'rubert', True, False, False, True),
        ('text-only', 'w2v', True, False, False, True),
        ('text-only', 'tfidf', True, False, False, True),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='XGBRegressor',
            model_params=xgbr_params,
            experiment_name="price_prediction",
            data_path=str(repo_path/'experiments'/'sample_300.csv'), # путь к короткому доку - если нужен полный
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            combinations=combinations,
            n_trials=1, # для полного воспроизведения тестов = 50
            sample_size=300, # для полного воспроизведения тестов = 65000
            subsample=150 # для полного воспроизведения тестов = 20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/XGBR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

2025-05-31 23:40:11,329 - PricePredictionExperiment_XGBRegressor - INFO - → Запуск конфигурации: price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_
INFO:PricePredictionExperiment_XGBRegressor:→ Запуск конфигурации: price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_
2025-05-31 23:40:11,331 - PricePredictionExperiment_XGBRegressor - INFO - Loading and preprocessing data...
INFO:PricePredictionExperiment_XGBRegressor:Loading and preprocessing data...
2025-05-31 23:40:11,364 - PricePredictionExperiment_XGBRegressor - INFO - Applying sample_size: 300
INFO:PricePredictionExperiment_XGBRegressor:Applying sample_size: 300
2025-05-31 23:40:11,373 - PricePredictionExperiment_XGBRegressor - INFO - Данные прошли строгую валидацию профиля.
INFO:PricePredictionExperiment_XGBRegressor:Данные прошли строгую валидацию профиля.


['landCategory', 'wallMaterial', 'bathroom', 'renovation', 'electricity', 'heating', 'waterSupply', 'gas', 'sewerage', 'media', 'parking', 'transportAccessibility', 'saleMethod', 'seller', 'category', 'infrastructure', 'terrace', 'recreation', 'region', 'nearestPPLA', 'nearestPPLA2']


[I 2025-05-31 23:40:26,424] A new study created in memory with name: no-name-b9b7e027-57df-4dc3-8ae4-4e15162d4e75
2025-05-31 23:40:26,630 - PricePredictionExperiment_XGBRegressor - INFO - Timing started for: total
INFO:PricePredictionExperiment_XGBRegressor:Timing started for: total
2025-05-31 23:40:26,643 - PricePredictionExperiment_XGBRegressor - INFO - ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
INFO:PricePredictionExperiment_XGBRegressor:ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
2025-05-31 23:40:26,654 - PricePredictionExperiment_XGBRegressor - INFO - ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:PricePredictionExperiment_XGBRegressor:ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
2025-05-31 23:40:26,666 - PricePredictionExperiment_XGBRegressor - INFO - ENV: sklearn_version: 1.6.1
INFO:PricePredictionExperiment_XGBRegressor:ENV: sklearn_version: 1.6.1
2025-05-31 23:40:26,678 - PricePredictionExperiment_XGBReg

CV stratified bins counts: [51 49 50]
Running 3-fold CV on 150 samples
 Fold 0: train_size=100 val_size=50


2025-05-31 23:40:27,352 - PricePredictionExperiment_XGBRegressor - INFO - Before reverse - price mean: 0.46163744810703816
INFO:PricePredictionExperiment_XGBRegressor:Before reverse - price mean: 0.46163744810703816
2025-05-31 23:40:27,356 - PricePredictionExperiment_XGBRegressor - INFO - After reverse - price mean: 52492874.37999999
INFO:PricePredictionExperiment_XGBRegressor:After reverse - price mean: 52492874.37999999
2025-05-31 23:40:27,410 - PricePredictionExperiment_XGBRegressor - INFO - Fold 0 - Log scale: R²=0.8970, RMSE=0.0702, SMAPE=13.9340%
Original scale: R²=0.4935, RMSE=87561917.5517, SMAPE=29.8956%
INFO:PricePredictionExperiment_XGBRegressor:Fold 0 - Log scale: R²=0.8970, RMSE=0.0702, SMAPE=13.9340%
Original scale: R²=0.4935, RMSE=87561917.5517, SMAPE=29.8956%


...

2025-05-31 23:48:24,136 - PricePredictionExperiment_XGBRegressor - INFO - [before_final_fit] CPU: 3.4%, RAM: 3.51GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
INFO:PricePredictionExperiment_XGBRegressor:[before_final_fit] CPU: 3.4%, RAM: 3.51GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
2025-05-31 23:48:31,244 - PricePredictionExperiment_XGBRegressor - INFO - [after_final_fit] CPU: 0.0%, RAM: 3.53GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
INFO:PricePredictionExperiment_XGBRegressor:[after_final_fit] CPU: 0.0%, RAM: 3.53GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB


[EXTRACT] Найден transformer под именем 'text': <class 'sklearn.pipeline.Pipeline'>
[SAVE] No tokenizer to save.
[SAVE] All available components saved to /content/results/price_prediction_XGBRegressor_text-only_w2v_pca_True_gate_False_mtf_False_ (logged to MLflow: True)


Registered model 'XGBRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False' already exists. Creating a new version of this model...
Created version '6' of model 'XGBRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False'.
2025-05-31 23:48:31,883 - PricePredictionExperiment_XGBRegressor - INFO - Model registered as 'XGBRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False', version 6
INFO:PricePredictionExperiment_XGBRegressor:Model registered as 'XGBRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False', version 6
2025-05-31 23:48:31,948 - PricePredictionExperiment_XGBRegressor - INFO - Full model saved to /content/results/price_prediction_XGBRegressor_text-only_w2v_pca_True_gate_False_mtf_False_ and logged to MLflow
INFO:PricePredictionExperiment_XGBRegressor:Full model saved to /content/results/price_prediction_XGBRegressor_text-only_w2v_pca_True_gate_False_mtf_False_ and logged to MLflow
2025-05-31 23:48:31,955 - PricePredictionExperiment_XGBRegressor 

""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### RFR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)
    params = {
        'model__bootstrap': [True, False],
        'model__max_depth': [10, 20, 30, None],
        'model__max_features': ['sqrt', 'log2'],  # Removed 'auto' for newer sklearn
        'model__min_samples_leaf': list(range(2, 26, 2)),
        'model__min_samples_split': list(range(2, 26, 2)),
        'model__n_estimators': [32, 64, 100, 500]
    }

    combinations = [
        ('categorical-only', None, False, False, True, True),
        ('categorical-only', None, False, False, False, True),
        ('mixed', 'rubert', True, False, False, True),
        ('mixed', 'w2v', True, False, False, True),
        ('mixed', 'tfidf', True, False, False, True),
        ('text-only', 'rubert', True, False, False, True),
        ('text-only', 'w2v', True, False, False, True),
        ('text-only', 'tfidf', True, False, False, True),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='RandomForestRegressor',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(repo_path/'experiments'/'sample_300.csv'), # путь к короткому доку - если нужен полный
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            combinations=combinations,
            n_trials=1, # для полного воспроизведения тестов = 50
            sample_size=300, # для полного воспроизведения тестов = 65000
            subsample=150 # для полного воспроизведения тестов = 20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/RFR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

2025-05-31 23:48:53,669 - PricePredictionExperiment_RandomForestRegressor - INFO - → Запуск конфигурации: price_prediction_RandomForestRegressor_None_none_pca_None_gate_None_mtf_True_
INFO:PricePredictionExperiment_RandomForestRegressor:→ Запуск конфигурации: price_prediction_RandomForestRegressor_None_none_pca_None_gate_None_mtf_True_
2025-05-31 23:48:53,671 - PricePredictionExperiment_RandomForestRegressor - INFO - Loading and preprocessing data...
INFO:PricePredictionExperiment_RandomForestRegressor:Loading and preprocessing data...
2025-05-31 23:48:53,694 - PricePredictionExperiment_RandomForestRegressor - INFO - Applying sample_size: 300
INFO:PricePredictionExperiment_RandomForestRegressor:Applying sample_size: 300
2025-05-31 23:48:53,701 - PricePredictionExperiment_RandomForestRegressor - INFO - Данные прошли строгую валидацию профиля.
INFO:PricePredictionExperiment_RandomForestRegressor:Данные прошли строгую валидацию профиля.


['landCategory', 'wallMaterial', 'bathroom', 'renovation', 'electricity', 'heating', 'waterSupply', 'gas', 'sewerage', 'media', 'parking', 'transportAccessibility', 'saleMethod', 'seller', 'category', 'infrastructure', 'terrace', 'recreation', 'region', 'nearestPPLA', 'nearestPPLA2']


[I 2025-05-31 23:49:06,427] A new study created in memory with name: no-name-f7ff68a3-6cdf-4ccf-9572-4a91b88c39c1
2025-05-31 23:49:06,648 - PricePredictionExperiment_RandomForestRegressor - INFO - Timing started for: total
INFO:PricePredictionExperiment_RandomForestRegressor:Timing started for: total
2025-05-31 23:49:06,661 - PricePredictionExperiment_RandomForestRegressor - INFO - ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
INFO:PricePredictionExperiment_RandomForestRegressor:ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
2025-05-31 23:49:06,673 - PricePredictionExperiment_RandomForestRegressor - INFO - ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:PricePredictionExperiment_RandomForestRegressor:ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
2025-05-31 23:49:06,684 - PricePredictionExperiment_RandomForestRegressor - INFO - ENV: sklearn_version: 1.6.1
INFO:PricePredictionExperiment_RandomForestRegressor:ENV: sklearn_v

CV stratified bins counts: [51 49 50]
Running 3-fold CV on 150 samples
 Fold 0: train_size=100 val_size=50
 Fold 1: train_size=100 val_size=50


2025-05-31 23:49:07,262 - PricePredictionExperiment_RandomForestRegressor - INFO - Fold 1 - Log scale: R²=0.6052, RMSE=0.1173, SMAPE=23.2217%
Original scale: R²=0.2312, RMSE=51034102.0636, SMAPE=60.9114%
INFO:PricePredictionExperiment_RandomForestRegressor:Fold 1 - Log scale: R²=0.6052, RMSE=0.1173, SMAPE=23.2217%
Original scale: R²=0.2312, RMSE=51034102.0636, SMAPE=60.9114%
2025-05-31 23:49:07,303 - PricePredictionExperiment_RandomForestRegressor - INFO - Before reverse - price mean: 0.48160117863933743
INFO:PricePredictionExperiment_RandomForestRegressor:Before reverse - price mean: 0.48160117863933743
2025-05-31 23:49:07,307 - PricePredictionExperiment_RandomForestRegressor - INFO - After reverse - price mean: 70292049.59999996
INFO:PricePredictionExperiment_RandomForestRegressor:After reverse - price mean: 70292049.59999996
2025-05-31 23:49:07,357 - PricePredictionExperiment_RandomForestRegressor - INFO - Fold 2 - Log scale: R²=0.5296, RMSE=0.1529, SMAPE=24.0376%
Original scale: R²

...

2025-05-31 23:54:35,299 - PricePredictionExperiment_RandomForestRegressor - INFO - [before_final_fit] CPU: 9.8%, RAM: 3.60GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
INFO:PricePredictionExperiment_RandomForestRegressor:[before_final_fit] CPU: 9.8%, RAM: 3.60GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
2025-05-31 23:54:36,167 - PricePredictionExperiment_RandomForestRegressor - INFO - [after_final_fit] CPU: 3.3%, RAM: 3.60GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB
INFO:PricePredictionExperiment_RandomForestRegressor:[after_final_fit] CPU: 3.3%, RAM: 3.60GB/12.67GB, GPU: 0%, GPU RAM: 1.06/15.00GB


[EXTRACT] Найден transformer под именем 'text': <class 'sklearn.pipeline.Pipeline'>
[SAVE] No tokenizer to save.
[SAVE] All available components saved to /content/results/price_prediction_RandomForestRegressor_text-only_w2v_pca_True_gate_False_mtf_False_ (logged to MLflow: True)


Registered model 'RandomForestRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False' already exists. Creating a new version of this model...
Created version '2' of model 'RandomForestRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False'.
2025-05-31 23:54:36,816 - PricePredictionExperiment_RandomForestRegressor - INFO - Model registered as 'RandomForestRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False', version 2
INFO:PricePredictionExperiment_RandomForestRegressor:Model registered as 'RandomForestRegressor+sample_300+tfidf+pca_True+gate_False+textfeat_False', version 2
2025-05-31 23:54:36,869 - PricePredictionExperiment_RandomForestRegressor - INFO - Full model saved to /content/results/price_prediction_RandomForestRegressor_text-only_w2v_pca_True_gate_False_mtf_False_ and logged to MLflow
INFO:PricePredictionExperiment_RandomForestRegressor:Full model saved to /content/results/price_prediction_RandomForestRegressor_text-only_w2v_pca_True_gate_False_mtf_Fal

""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### ANN experiment initiation


In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    params = {
        'model__learning_rate': [0.0005, 0.001, 0.002, 0.003],  # сужаем диапазон
        'model__neurons_layer1': [100, 128, 150, 180, 200],
        'model__neurons_layer2': [40, 64, 80, 100],
        'model__neurons_layer3': [16, 32, 48, 64],
        'model__batch_size': [64, 96, 128],  # убираем очень маленькие батчи
        'model__epochs': [70, 100, 120],  # увеличиваем верхнюю границу
        'model__l1_reg': [0.0005, 0.001, 0.005, 0.01],
        'model__l2_reg': [0.0005, 0.001, 0.005, 0.01],
        'model__activation': ['sigmoid']  # оставляем только sigmoid
    }

    combinations = [
        ('categorical-only', None, False, False, True, False),
        ('categorical-only', None, False, False, False, False),
        ('mixed', 'rubert', True, False, False, False),
        ('mixed', 'w2v', True, False, False, False),
        ('mixed', 'tfidf', True, False, False, False),
        ('text-only', 'rubert', True, False, False, False),
        ('text-only', 'w2v', True, False, False, False),
        ('text-only', 'tfidf', True, False, False, False),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='ANNRegressor',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(repo_path/'experiments'/'sample_300.csv'), # путь к короткому доку - если нужен полный
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            combinations=combinations,
            n_trials=1, # для полного воспроизведения тестов = 50
            sample_size=300, # для полного воспроизведения тестов = 65000
            subsample=150 # для полного воспроизведения тестов = 20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/ANN'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

2025-05-31 23:54:53,861 - PricePredictionExperiment_ANNRegressor - INFO - → Запуск конфигурации: price_prediction_ANNRegressor_None_none_pca_None_gate_None_mtf_True_
INFO:PricePredictionExperiment_ANNRegressor:→ Запуск конфигурации: price_prediction_ANNRegressor_None_none_pca_None_gate_None_mtf_True_
2025-05-31 23:54:53,866 - PricePredictionExperiment_ANNRegressor - INFO - Loading and preprocessing data...
INFO:PricePredictionExperiment_ANNRegressor:Loading and preprocessing data...
2025-05-31 23:54:53,908 - PricePredictionExperiment_ANNRegressor - INFO - Applying sample_size: 300
INFO:PricePredictionExperiment_ANNRegressor:Applying sample_size: 300
2025-05-31 23:54:53,922 - PricePredictionExperiment_ANNRegressor - INFO - Данные прошли строгую валидацию профиля.
INFO:PricePredictionExperiment_ANNRegressor:Данные прошли строгую валидацию профиля.


['landCategory', 'wallMaterial', 'bathroom', 'renovation', 'electricity', 'heating', 'waterSupply', 'gas', 'sewerage', 'media', 'parking', 'transportAccessibility', 'saleMethod', 'seller', 'category', 'infrastructure', 'terrace', 'recreation', 'region', 'nearestPPLA', 'nearestPPLA2']


[I 2025-05-31 23:55:07,461] A new study created in memory with name: no-name-76dbb481-d3ed-4588-8e52-33070dda7b50
2025-05-31 23:55:07,709 - PricePredictionExperiment_ANNRegressor - INFO - Timing started for: total
INFO:PricePredictionExperiment_ANNRegressor:Timing started for: total
2025-05-31 23:55:07,722 - PricePredictionExperiment_ANNRegressor - INFO - ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
INFO:PricePredictionExperiment_ANNRegressor:ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
2025-05-31 23:55:07,735 - PricePredictionExperiment_ANNRegressor - INFO - ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:PricePredictionExperiment_ANNRegressor:ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
2025-05-31 23:55:07,747 - PricePredictionExperiment_ANNRegressor - INFO - ENV: sklearn_version: 1.6.1
INFO:PricePredictionExperiment_ANNRegressor:ENV: sklearn_version: 1.6.1
2025-05-31 23:55:07,758 - PricePredictionExperiment_ANNReg

CV stratified bins counts: [51 49 50]
Running 3-fold CV on 150 samples
 Fold 0: train_size=100 val_size=50


2025-05-31 23:55:08,376 - PricePredictionExperiment_ANNRegressor - INFO - Before reverse - price mean: 0.46163744810703816
INFO:PricePredictionExperiment_ANNRegressor:Before reverse - price mean: 0.46163744810703816
2025-05-31 23:55:08,382 - PricePredictionExperiment_ANNRegressor - INFO - After reverse - price mean: 52492874.37999999
INFO:PricePredictionExperiment_ANNRegressor:After reverse - price mean: 52492874.37999999
2025-05-31 23:55:08,441 - PricePredictionExperiment_ANNRegressor - INFO - Fold 0 - Log scale: R²=-0.9628, RMSE=0.3063, SMAPE=54.2527%
Original scale: R²=0.0432, RMSE=120348523.7328, SMAPE=139.8799%
INFO:PricePredictionExperiment_ANNRegressor:Fold 0 - Log scale: R²=-0.9628, RMSE=0.3063, SMAPE=54.2527%
Original scale: R²=0.0432, RMSE=120348523.7328, SMAPE=139.8799%
2025-05-31 23:55:08,506 - PricePredictionExperiment_ANNRegressor - INFO - Before reverse - price mean: 0.44923494890588567
INFO:PricePredictionExperiment_ANNRegressor:Before reverse - price mean: 0.4492349489

...

2025-05-31 23:58:27,430 - PricePredictionExperiment_ANNRegressor - INFO - Before reverse - price mean: 0.4580297314108511
INFO:PricePredictionExperiment_ANNRegressor:Before reverse - price mean: 0.4580297314108511
2025-05-31 23:58:27,438 - PricePredictionExperiment_ANNRegressor - INFO - After reverse - price mean: 63415391.4893617
INFO:PricePredictionExperiment_ANNRegressor:After reverse - price mean: 63415391.4893617
2025-05-31 23:58:27,444 - PricePredictionExperiment_ANNRegressor - INFO -  Fold 4/5 | Log R2=-0.0369 | Orig R2=-0.0933
INFO:PricePredictionExperiment_ANNRegressor: Fold 4/5 | Log R2=-0.0369 | Orig R2=-0.0933


 FinalCV Fold 4: train_size=189 val_size=47


2025-05-31 23:58:28,279 - PricePredictionExperiment_ANNRegressor - INFO - Before reverse - price mean: 0.4727362701111664
INFO:PricePredictionExperiment_ANNRegressor:Before reverse - price mean: 0.4727362701111664
2025-05-31 23:58:28,287 - PricePredictionExperiment_ANNRegressor - INFO - After reverse - price mean: 67053424.65957447
INFO:PricePredictionExperiment_ANNRegressor:After reverse - price mean: 67053424.65957447
2025-05-31 23:58:28,294 - PricePredictionExperiment_ANNRegressor - INFO -  Fold 5/5 | Log R2=-0.0020 | Orig R2=-0.1574
INFO:PricePredictionExperiment_ANNRegressor: Fold 5/5 | Log R2=-0.0020 | Orig R2=-0.1574
2025-05-31 23:58:29,055 - PricePredictionExperiment_ANNRegressor - INFO - Before reverse - price mean: 0.46124991486493366
INFO:PricePredictionExperiment_ANNRegressor:Before reverse - price mean: 0.46124991486493366
2025-05-31 23:58:29,059 - PricePredictionExperiment_ANNRegressor - INFO - After reverse - price mean: 53129052.52542374
INFO:PricePredictionExperiment_A

""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### LSVR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    params = {
        'model__C': [5, 10, 20, 50],  # Focus around the best found value
        'model__max_iter': [5000, 7000, 10000],  # Higher values didn't hurt
        'model__dual': [False],  # Only this worked well
        'model__loss': ['squared_epsilon_insensitive']  # Only compatible with dual=False
    }

    combinations = [
        ('categorical-only', None, False, False, True, False),
        ('categorical-only', None, False, False, False, False),
        ('mixed', 'rubert', True, False, False, False),
        ('mixed', 'w2v', True, False, False, False),
        ('mixed', 'tfidf', True, False, False, False),
        ('text-only', 'rubert', True, False, False, False),
        ('text-only', 'w2v', True, False, False, False),
        ('text-only', 'tfidf', True, False, False, False),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='LinearSVR',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(repo_path/'experiments'/'sample_300.csv'), # путь к короткому доку - если нужен полный
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            combinations=combinations,
            n_trials=1, # для полного воспроизведения тестов = 50
            sample_size=300, # для полного воспроизведения тестов = 65000
            subsample=150 # для полного воспроизведения тестов = 20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/LinearSVR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

2025-05-31 23:59:12,660 - PricePredictionExperiment_LinearSVR - INFO - → Запуск конфигурации: price_prediction_LinearSVR_None_none_pca_None_gate_None_mtf_True_
INFO:PricePredictionExperiment_LinearSVR:→ Запуск конфигурации: price_prediction_LinearSVR_None_none_pca_None_gate_None_mtf_True_
2025-05-31 23:59:12,664 - PricePredictionExperiment_LinearSVR - INFO - Loading and preprocessing data...
INFO:PricePredictionExperiment_LinearSVR:Loading and preprocessing data...
2025-05-31 23:59:12,697 - PricePredictionExperiment_LinearSVR - INFO - Applying sample_size: 300
INFO:PricePredictionExperiment_LinearSVR:Applying sample_size: 300
2025-05-31 23:59:12,704 - PricePredictionExperiment_LinearSVR - INFO - Данные прошли строгую валидацию профиля.
INFO:PricePredictionExperiment_LinearSVR:Данные прошли строгую валидацию профиля.


['landCategory', 'wallMaterial', 'bathroom', 'renovation', 'electricity', 'heating', 'waterSupply', 'gas', 'sewerage', 'media', 'parking', 'transportAccessibility', 'saleMethod', 'seller', 'category', 'infrastructure', 'terrace', 'recreation', 'region', 'nearestPPLA', 'nearestPPLA2']


[I 2025-05-31 23:59:24,998] A new study created in memory with name: no-name-e966ec34-1ebd-46ef-aec7-6adbc28ece50
2025-05-31 23:59:25,214 - PricePredictionExperiment_LinearSVR - INFO - Timing started for: total
INFO:PricePredictionExperiment_LinearSVR:Timing started for: total
2025-05-31 23:59:25,226 - PricePredictionExperiment_LinearSVR - INFO - ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
INFO:PricePredictionExperiment_LinearSVR:ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
2025-05-31 23:59:25,238 - PricePredictionExperiment_LinearSVR - INFO - ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:PricePredictionExperiment_LinearSVR:ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
2025-05-31 23:59:25,261 - PricePredictionExperiment_LinearSVR - INFO - ENV: sklearn_version: 1.6.1
INFO:PricePredictionExperiment_LinearSVR:ENV: sklearn_version: 1.6.1
2025-05-31 23:59:25,274 - PricePredictionExperiment_LinearSVR - INFO - ENV: pandas

CV stratified bins counts: [51 49 50]
Running 3-fold CV on 150 samples
 Fold 0: train_size=100 val_size=50
 Fold 1: train_size=100 val_size=50
 Fold 2: train_size=100 val_size=50


2025-05-31 23:59:25,864 - PricePredictionExperiment_LinearSVR - INFO - Fold 2 - Log scale: R²=0.5209, RMSE=0.1543, SMAPE=15.9799%
Original scale: R²=0.5425, RMSE=102532226.9254, SMAPE=39.8297%
INFO:PricePredictionExperiment_LinearSVR:Fold 2 - Log scale: R²=0.5209, RMSE=0.1543, SMAPE=15.9799%
Original scale: R²=0.5425, RMSE=102532226.9254, SMAPE=39.8297%
[I 2025-05-31 23:59:25,867] Trial 0 finished with value: 0.3747320451042107 and parameters: {'model__C': 22, 'model__max_iter': 9754, 'model__dual': False, 'model__loss': 'squared_epsilon_insensitive'}. Best is trial 0 with value: 0.3747320451042107.
2025-05-31 23:59:25,868 - PricePredictionExperiment_LinearSVR - INFO - Elapsed time for optuna: 0.43 seconds
INFO:PricePredictionExperiment_LinearSVR:Elapsed time for optuna: 0.43 seconds
2025-05-31 23:59:25,880 - PricePredictionExperiment_LinearSVR - INFO - Timing started for: final_cv
INFO:PricePredictionExperiment_LinearSVR:Timing started for: final_cv
2025-05-31 23:59:25,970 - PricePred

...

2025-06-01 00:03:02,276 - PricePredictionExperiment_LinearSVR - INFO - Before reverse - price mean: 0.4580297314108511
INFO:PricePredictionExperiment_LinearSVR:Before reverse - price mean: 0.4580297314108511
2025-06-01 00:03:02,279 - PricePredictionExperiment_LinearSVR - INFO - After reverse - price mean: 63415391.4893617
INFO:PricePredictionExperiment_LinearSVR:After reverse - price mean: 63415391.4893617
2025-06-01 00:03:02,290 - PricePredictionExperiment_LinearSVR - INFO -  Fold 4/5 | Log R2=0.7071 | Orig R2=0.5807
INFO:PricePredictionExperiment_LinearSVR: Fold 4/5 | Log R2=0.7071 | Orig R2=0.5807


 FinalCV Fold 4: train_size=189 val_size=47


2025-06-01 00:03:02,607 - PricePredictionExperiment_LinearSVR - INFO - Before reverse - price mean: 0.4727362701111664
INFO:PricePredictionExperiment_LinearSVR:Before reverse - price mean: 0.4727362701111664
2025-06-01 00:03:02,610 - PricePredictionExperiment_LinearSVR - INFO - After reverse - price mean: 67053424.65957447
INFO:PricePredictionExperiment_LinearSVR:After reverse - price mean: 67053424.65957447
2025-06-01 00:03:02,618 - PricePredictionExperiment_LinearSVR - INFO -  Fold 5/5 | Log R2=0.6217 | Orig R2=0.2151
INFO:PricePredictionExperiment_LinearSVR: Fold 5/5 | Log R2=0.6217 | Orig R2=0.2151
2025-06-01 00:03:03,044 - PricePredictionExperiment_LinearSVR - INFO - Before reverse - price mean: 0.46124991486493366
INFO:PricePredictionExperiment_LinearSVR:Before reverse - price mean: 0.46124991486493366
2025-06-01 00:03:03,051 - PricePredictionExperiment_LinearSVR - INFO - After reverse - price mean: 53129052.52542374
INFO:PricePredictionExperiment_LinearSVR:After reverse - price 

""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Initiation for full experiment repetition on full march dataset

In [ ]:
# !pip install -U gdown  # Обновляем gdown до последней версии

In [ ]:
# получаем данные за март и сохраняем к себе чтобы применять в тестах

import pandas as pd

# 1. Указываем ID файла из Google Drive
file_id = '14k-_OHJCHsaB1d6xOY07FeHdrXEVxdx3'

# 2. Скачиваем файл с помощью gdown (новая версия)
!gdown "https://drive.google.com/uc?id={file_id}" -O data.csv

# 3. Проверяем размер файла
!ls -lh data.csv

# 4. Загружаем в DataFrame (с обработкой возможных ошибок)
try:
    df = pd.read_csv('data.csv')
    print("Данные успешно загружены:")
    print(f"Строк: {len(df)}, колонок: {len(df.columns)}")
    print(df.head())
except Exception as e:
    print(f"Ошибка при чтении CSV: {str(e)}")
    print("Попробуйте указать кодировку или разделитель вручную:")
    df = pd.read_csv('data.csv', encoding='utf-8', sep=',')  # или encoding='windows-1251'

print("Данные успешно загружены:")

path = 'some_path.csv'
df.set_index('id').to_csv(path)

Downloading...
From (original): https://drive.google.com/uc?id=14k-_OHJCHsaB1d6xOY07FeHdrXEVxdx3
From (redirected): https://drive.google.com/uc?id=14k-_OHJCHsaB1d6xOY07FeHdrXEVxdx3&confirm=t&uuid=9073dd7d-aa89-49ea-9935-d7eefb0ddb65
To: /content/data.csv
100% 316M/316M [00:01<00:00, 244MB/s]
-rw-r--r-- 1 root root 302M May 31 23:25 data.csv
Данные успешно загружены:
Строк: 69439, колонок: 33
           id Количество комнат  Площадь дома  Площадь участка Этажей в доме  \
0   109147246                 5         210.0             20.0             2   
1   749921374                 8         253.0             27.0             3   
2  4014924213                 4         150.0              8.5             2   
3  1029200202       10 и больше         450.0              9.0    4 и больше   
4  1064304410                 2          25.0             20.0             1   

                              Категория земель  Год постройки Материал стен  \
0  индивидуальное жилищное строительство (ИЖС

### XGBR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    xgbr_params = {
        'model__n_estimators': [500, 1000],
        'model__max_depth': [4, 6],
        'model__learning_rate': [0.005, 0.01, 0.02, 0.05],
        'model__subsample': [0.8, 1.0]
    }

    combinations = [
        ('categorical-only', None, False, False, True, True),
        ('categorical-only', None, False, False, False, True),
        ('mixed', 'rubert', True, False, False, True),
        ('mixed', 'w2v', True, False, False, True),
        ('mixed', 'tfidf', True, False, False, True),
        ('text-only', 'rubert', True, False, False, True),
        ('text-only', 'w2v', True, False, False, True),
        ('text-only', 'tfidf', True, False, False, True),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='XGBRegressor',
            model_params=xgbr_params,
            experiment_name="price_prediction",
            data_path=str(path), # путь к короткому доку куда был скачан полный файл
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            combinations=combinations,
            n_trials=50,
            sample_size=65000,
            subsample=20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/XGBR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

2025-06-01 00:08:33,686 - PricePredictionExperiment_XGBRegressor - INFO - → Запуск конфигурации: price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_
INFO:PricePredictionExperiment_XGBRegressor:→ Запуск конфигурации: price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_
2025-06-01 00:08:33,688 - PricePredictionExperiment_XGBRegressor - INFO - Loading and preprocessing data...
INFO:PricePredictionExperiment_XGBRegressor:Loading and preprocessing data...
2025-06-01 00:08:39,974 - PricePredictionExperiment_XGBRegressor - INFO - Applying sample_size: 65000
INFO:PricePredictionExperiment_XGBRegressor:Applying sample_size: 65000
2025-06-01 00:08:40,144 - PricePredictionExperiment_XGBRegressor - INFO - Данные прошли строгую валидацию профиля.
INFO:PricePredictionExperiment_XGBRegressor:Данные прошли строгую валидацию профиля.


['landCategory', 'wallMaterial', 'bathroom', 'renovation', 'electricity', 'heating', 'waterSupply', 'gas', 'sewerage', 'media', 'parking', 'transportAccessibility', 'saleMethod', 'seller', 'category', 'infrastructure', 'terrace', 'recreation', 'region', 'nearestPPLA', 'nearestPPLA2']


[I 2025-06-01 00:10:29,105] A new study created in memory with name: no-name-b7df85b4-e4be-4ba2-97b5-dd94d02718fb
2025-06-01 00:10:29,327 - PricePredictionExperiment_XGBRegressor - INFO - Timing started for: total
INFO:PricePredictionExperiment_XGBRegressor:Timing started for: total
2025-06-01 00:10:29,338 - PricePredictionExperiment_XGBRegressor - INFO - ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
INFO:PricePredictionExperiment_XGBRegressor:ENV: python_version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
2025-06-01 00:10:29,351 - PricePredictionExperiment_XGBRegressor - INFO - ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:PricePredictionExperiment_XGBRegressor:ENV: platform: Linux-6.1.123+-x86_64-with-glibc2.35
2025-06-01 00:10:29,364 - PricePredictionExperiment_XGBRegressor - INFO - ENV: sklearn_version: 1.6.1
INFO:PricePredictionExperiment_XGBRegressor:ENV: sklearn_version: 1.6.1
2025-06-01 00:10:29,378 - PricePredictionExperiment_XGBReg

CV stratified bins counts: [6670 6663 6667]
Running 3-fold CV on 20000 samples
 Fold 0: train_size=13333 val_size=6667


2025-06-01 00:10:35,244 - PricePredictionExperiment_XGBRegressor - INFO - Before reverse - price mean: 0.6131231136815939
INFO:PricePredictionExperiment_XGBRegressor:Before reverse - price mean: 0.6131231136815939
2025-06-01 00:10:35,249 - PricePredictionExperiment_XGBRegressor - INFO - After reverse - price mean: 46994876.19484026
INFO:PricePredictionExperiment_XGBRegressor:After reverse - price mean: 46994876.19484026
2025-06-01 00:10:35,251 - PricePredictionExperiment_XGBRegressor - WARNING - Unrealistic price values detected: min=1640.00, max=3119770000.00
2025-06-01 00:10:35,319 - PricePredictionExperiment_XGBRegressor - INFO - Fold 0 - Log scale: R²=0.9713, RMSE=0.0155, SMAPE=1.2022%
Original scale: R²=0.8757, RMSE=48069597.5932, SMAPE=10.8863%
INFO:PricePredictionExperiment_XGBRegressor:Fold 0 - Log scale: R²=0.9713, RMSE=0.0155, SMAPE=1.2022%
Original scale: R²=0.8757, RMSE=48069597.5932, SMAPE=10.8863%


...

2025-06-01 00:28:35,578 - PricePredictionExperiment_XGBRegressor - INFO - [before_final_fit] CPU: 0.0%, RAM: 4.71GB/12.67GB, GPU: 0%, GPU RAM: 1.16/15.00GB
INFO:PricePredictionExperiment_XGBRegressor:[before_final_fit] CPU: 0.0%, RAM: 4.71GB/12.67GB, GPU: 0%, GPU RAM: 1.16/15.00GB
2025-06-01 00:28:58,574 - PricePredictionExperiment_XGBRegressor - INFO - [after_final_fit] CPU: 1.7%, RAM: 4.81GB/12.67GB, GPU: 0%, GPU RAM: 1.16/15.00GB
INFO:PricePredictionExperiment_XGBRegressor:[after_final_fit] CPU: 1.7%, RAM: 4.81GB/12.67GB, GPU: 0%, GPU RAM: 1.16/15.00GB


[EXTRACT] В ColumnTransformer нет шага 'text'
[SAVE] No transformer to save.
[SAVE] No tokenizer to save.
[SAVE] All available components saved to /content/results/price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_ (logged to MLflow: True)


Successfully registered model 'XGBRegressor+some_path+None+pca_False+gate_False+textfeat_True'.
Created version '1' of model 'XGBRegressor+some_path+None+pca_False+gate_False+textfeat_True'.
2025-06-01 00:28:59,148 - PricePredictionExperiment_XGBRegressor - INFO - Model registered as 'XGBRegressor+some_path+None+pca_False+gate_False+textfeat_True', version 1
INFO:PricePredictionExperiment_XGBRegressor:Model registered as 'XGBRegressor+some_path+None+pca_False+gate_False+textfeat_True', version 1
2025-06-01 00:28:59,202 - PricePredictionExperiment_XGBRegressor - INFO - Full model saved to /content/results/price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_ and logged to MLflow
INFO:PricePredictionExperiment_XGBRegressor:Full model saved to /content/results/price_prediction_XGBRegressor_None_none_pca_None_gate_None_mtf_True_ and logged to MLflow
2025-06-01 00:28:59,216 - PricePredictionExperiment_XGBRegressor - INFO - Elapsed time for final_model_teaching: 132.78 seconds

""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### RFR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    params = {
        'model__bootstrap': [True, False],
        'model__max_depth': [10, 20, 30, None],
        'model__max_features': ['sqrt', 'log2'],  # Removed 'auto' for newer sklearn
        'model__min_samples_leaf': list(range(2, 26, 2)),
        'model__min_samples_split': list(range(2, 26, 2)),
        'model__n_estimators': [32, 64, 100, 500]
    }

    combinations = [
        ('categorical-only', None, False, False, True, True),
        ('categorical-only', None, False, False, False, True),
        ('mixed', 'rubert', True, False, False, True),
        ('mixed', 'w2v', True, False, False, True),
        ('mixed', 'tfidf', True, False, False, True),
        ('text-only', 'rubert', True, False, False, True),
        ('text-only', 'w2v', True, False, False, True),
        ('text-only', 'tfidf', True, False, False, True),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='RandomForestRegressor',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(path), # путь к короткому доку куда был скачан полный файл
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            n_trials=50,
            sample_size=65000,
            subsample=20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/RFR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

### ANN experiment initiation


In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    params = {
        'model__learning_rate': [0.0005, 0.001, 0.002, 0.003],  # сужаем диапазон
        'model__neurons_layer1': [100, 128, 150, 180, 200],
        'model__neurons_layer2': [40, 64, 80, 100],
        'model__neurons_layer3': [16, 32, 48, 64],
        'model__batch_size': [64, 96, 128],  # убираем очень маленькие батчи
        'model__epochs': [70, 100, 120],  # увеличиваем верхнюю границу
        'model__l1_reg': [0.0005, 0.001, 0.005, 0.01],
        'model__l2_reg': [0.0005, 0.001, 0.005, 0.01],
        'model__activation': ['sigmoid']  # оставляем только sigmoid
    }

    combinations = [
        ('categorical-only', None, False, False, True, False),
        ('categorical-only', None, False, False, False, False),
        ('mixed', 'rubert', True, False, False, False),
        ('mixed', 'w2v', True, False, False, False),
        ('mixed', 'tfidf', True, False, False, False),
        ('text-only', 'rubert', True, False, False, False),
        ('text-only', 'w2v', True, False, False, False),
        ('text-only', 'tfidf', True, False, False, False),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='ANNRegressor',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(path), # путь к короткому доку куда был скачан полный файл
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            n_trials=50,
            sample_size=65000,
            subsample=20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/ANN'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise

### LSVR experiment initiation

In [ ]:
import logging
import pandas as pd
import json
from pathlib import Path

# 3. Создаем временную папку для результатов
!mkdir -p /content/results

# 4. Основной код эксперимента
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('experiment.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    params = {
        'model__C': [5, 10, 20, 50],  # Focus around the best found value
        'model__max_iter': [5000, 7000, 10000],  # Higher values didn't hurt
        'model__dual': [False],  # Only this worked well
        'model__loss': ['squared_epsilon_insensitive']  # Only compatible with dual=False
    }

    combinations = [
        ('categorical-only', None, False, False, True, False),
        ('categorical-only', None, False, False, False, False),
        ('mixed', 'rubert', True, False, False, False),
        ('mixed', 'w2v', True, False, False, False),
        ('mixed', 'tfidf', True, False, False, False),
        ('text-only', 'rubert', True, False, False, False),
        ('text-only', 'w2v', True, False, False, False),
        ('text-only', 'tfidf', True, False, False, False),
    ]

    try:
        # Полные пути к файлам в репозитории
        repo_path = Path('/content/Price-prediction-with-textual-data')

        experiment = PricePredictionExperiment(
            model_name='LinearSVR',
            model_params=params,
            experiment_name="price_prediction",
            data_path=str(path), # путь к короткому доку куда был скачан полный файл
            results_dir='/content/results',  # временная папка в Colab
            random_state=42,
            use_hex_features=True,
            hex_resolution=10,
            verbose=True,
            manual_text_params=True,
            manual_text_params_path=str(repo_path/'experiments'/'manual_text_features_negative.json'),
            profile_json_path=str(repo_path/'experiments'/'uniques_dict.json')
        )

        # Запуск экспериментов
        results = experiment.run_for_combinations(
            n_trials=50,
            sample_size=65000,
            subsample=20000
        )

        # Сохранение и отображение результатов
        results_df = experiment.create_results_table(list(results.keys()))
        display(results_df)

        # Сохраняем результаты в CSV
        results_df.to_csv('/content/results/experiment_results.csv', index=False)
        logger.info("Эксперимент успешно завершен!")

        # Копируем результаты в Google Drive (если нужно)
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r /content/results '/content/drive/MyDrive/thesis/DROP_SAMPLE/LinearSVR'
        logger.info("Результаты сохранены в Google Drive")

    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        raise